# Tarea para el Hogar 02

Esta Tarea para el Hogar 02 se entrega el final de la segunda clase
<br> se espera de usted que intente avanzar con los desafios propuestos y que los traiga terminados para la Clase 03, ya que se analizarán los resultados

##  1. Ensembles de Modelos

Vea el siguiente video [BBC - The Code - The Wisdom of the Crowd](https://www.youtube.com/watch?v=iOucwX7Z1HU)    ( 5 min)


Lea los siguientes artículos


*   [The Wisdom of Crowds (Vox Populi) by Francis Galton](https://www.all-about-psychology.com/the-wisdom-of-crowds.html)  (10 min)
*   [A Gentle Introduction to Ensemble Learning](https://machinelearningmastery.com/what-is-ensemble-learning/)  (10 min)





---



##  2.  Zero2Hero   primera parte
Se han lanzado los primeros fascículos coleccionables llamados "from Zero to Hero" que muy detalladamente, paso a paso enseñan todo lo necesario de R para entender los scripts oficiales de la asignatura.
Están en el repositorio oficial de la asignatura, carpeta  **src/zero2hero**



---



## 3.  Grid Search

Busque en internet el precido significado de los hiperparámetros de la librería **rpart**  que está implementando el algoritmo **CART**  Classification and Regression Trees  propuesto en el año 1984 por Leo Brieman:

*   cp
*   maxdepth
*   minsplit
*   minbucket

Entienda que valores es razonable tome cada hiperparámetro,  en particular profundice en el hiperparámetro  **cp**  y la posibilidad que tome valores negativos.  Es válido consultar a su amigo de *capacidades especiales*  ChatGPT


En las siguientes celdas a un notebook incompleto, un esqueleto de codigo brindado a modo de facilitarle la tarea de codeo y permitir que su valiosa cognición se concentre temas conceptuales de Ciencia de Datos

Modifiquelo agregando loops para que recorra TODOS los hiperparámetros de rpart  < cp, maxdepth, minsplit, minbucket >, y luego póngalo a correr. Recuerde cambiar por SU semilla
Tenga muy presente la granularidad que eligirá para cada hiperparámetro.

### Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi



cp: cannot stat '/content/buckets/b1/kaggle/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


limpio el ambiente de R

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,803464,43.0,1454471,77.7,1454471,77.7
Vcells,1504788,11.5,56777584,433.2,70971312,541.5


In [ ]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")
if (!require("primes")) install.packages("primes")
require("primes")

Aqui debe poner SU semiila primigenia

In [ ]:
PARAM <- list()
# reemplazar por su primer semilla
PARAM$semilla_primigenia <- 533317
PARAM$qsemillas <- 1

PARAM$training_pct <- 70L  # entre  1L y 99L

# elegir SU dataset comentando/ descomentando
PARAM$dataset_nom <- "~/datasets/dataset_pequeno.csv"

In [ ]:
# particionar agrega una columna llamada fold a un dataset
#  que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)
#   crea una particion 70, 30

particionar <- function(data, division, agrupa = "", campo = "fold", start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(function(x, y) {
    rep(y, x)
  }, division, seq(from = start, length.out = length(division))))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by = agrupa
  ]
}


In [ ]:
ArbolEstimarGanancia <- function(semilla, training_pct, param_basicos) {
  # particiono estratificadamente el dataset
  particionar(dataset,
    division = c(training_pct, 100L -training_pct),
    agrupa = "clase_ternaria",
    seed = semilla # aqui se usa SU semilla
  )

  # genero el modelo
  # predecir clase_ternaria a partir del resto
  modelo <- rpart("clase_ternaria ~ .",
    data = dataset[fold == 1], # fold==1  es training,  el 70% de los datos
    xval = 0,
    control = param_basicos
  ) # aqui van los parametros del arbol

  # aplico el modelo a los datos de testing
  prediccion <- predict(modelo, # el modelo que genere recien
    dataset[fold == 2], # fold==2  es testing, el 30% de los datos
    type = "prob"
  ) # type= "prob"  es que devuelva la probabilidad

  # prediccion es una matriz con TRES columnas,
  #  llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
  # cada columna es el vector de probabilidades


  # calculo la ganancia en testing  qu es fold==2
  ganancia_test <- dataset[
    fold == 2,
    sum(ifelse(prediccion[, "BAJA+2"] > 0.025,
      ifelse(clase_ternaria == "BAJA+2", 117000, -3000),
      0
    ))
  ]

  # escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada <- ganancia_test / (( 100 - PARAM$training_pct ) / 100 )

  return(
    c( list("semilla" = semilla),
      param_basicos,
      list( "ganancia_test" = ganancia_test_normalizada )
     )
  )
}


In [ ]:
ArbolesMontecarlo <- function(semillas, param_basicos) {

  # la funcion mcmapply  llama a la funcion ArbolEstimarGanancia
  #  tantas veces como valores tenga el vector  PARAM$semillas
  salida <- mcmapply(ArbolEstimarGanancia,
    semillas, # paso el vector de semillas
    MoreArgs = list(PARAM$training_pct, param_basicos), # aqui paso el segundo parametro
    SIMPLIFY = FALSE,
    mc.cores = detectCores()
  )

  return(salida)
}


In [ ]:
# carpeta de trabajo
# por fabor cambiar numero de experimento si se cambia el loop principal
setwd("/content/buckets/b1/exp")
experimento <- "HT2900"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

In [ ]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

# trabajo solo con los datos con clase, es decir 202107
dataset <- dataset[clase_ternaria != ""]

In [ ]:

# genero numeros primos
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia) # inicializo
# me quedo con PARAM$qsemillas   semillas
PARAM$semillas <- sample(primos, PARAM$qsemillas )


In [ ]:
# genero la data.table donde van los resultados detallados del Grid Search
# un registro para cada combinacion de < semilla, parametros >

if(file.exists("gridsearch_detalle.txt")){
  tb_grid_search_detalle <- fread("gridsearch_detalle.txt")
}else{
  tb_grid_search_detalle <- data.table(
    semilla = integer(),
    cp = numeric(),
    maxdepth = integer(),
   minsplit = integer(),
    minbucket = integer(),
    ganancia_test = numeric()
  )
}

nrow( tb_grid_search_detalle )

[1] 450

Esta es la parte del código que usted debe expandir a TODOS los hiperparámetros de rpart,
<br>ya que actualmente apenas recorre  maxdepth y  minsplit  dejando fijos  cp=-0.5  y minbucket=5

In [ ]:
# ======================================================
# GRID SEARCH con 4 hiperparámetros y misma salida que tu código
# - Mantiene: iter, cat/flush, checkpoint con PARAM$qsemillas,
#   acumulación en tb_grid_search_detalle y fwrite por loop externo (maxdepth)
# - Tamaño de grilla: 500 brutas -> 450 válidas (coherencia minbucket<=minsplit/2)
# ======================================================

iter <- 0

cp_vals        <- c(-0.001, -0.05, -0.01, 0, 0.001)   # 5 valores (incluye negativos)
maxdepth_vals  <- c(6, 8, 10, 12, 14)               # 5 valores
minsplit_vals  <- c(50, 100, 200, 400, 600)              # 4 valores
minbucket_vals <- c(5, 10, 20, 30, 40)              # 5 valores

for (vmax_depth in maxdepth_vals) {                       # loop externo = igual que tu código
  for (vcp in cp_vals) {                                  # ahora cp también barre
    for (vmin_split in minsplit_vals) {
      for (vmin_bucket in minbucket_vals) {

        # coherencia básica: si no se cumple, salteamos (esto deja 450 combinaciones)
        if (vmin_bucket > floor(vmin_split / 2)) next

        iter <- iter + 1
        cat(iter, " ")
        flush.console()

        # checkpoint para reanudar (idéntico al tuyo)
        if (iter * PARAM$qsemillas < nrow(tb_grid_search_detalle) + 1) next

        # armo lista de hiperparámetros (idéntico a tu esquema)
        param_basicos <- list(
          "cp"        = vcp,          # complejidad (negativos permitidos como pediste)
          "maxdepth"  = vmax_depth,   # profundidad máxima
          "minsplit"  = vmin_split,   # mínimo de registros para split
          "minbucket" = vmin_bucket   # mínimo de registros en hoja
        )

        # corre Montecarlo con tus semillas y devuelve data.tables ya uniformadas
        ganancias <- ArbolesMontecarlo(PARAM$semillas, param_basicos)

        # acumula resultados en la misma tabla de detalle (igual que tu código)
        tb_grid_search_detalle <- rbindlist(
          list(tb_grid_search_detalle, rbindlist(ganancias)),
          use.names = TRUE, fill = TRUE
        )
      }
    }
  }

  # grabar en el loop más externo (maxdepth), como en tu código original
  fwrite(tb_grid_search_detalle,
         file = "gridsearch_detalle.txt",
         sep = "\t")
}

cat("\nCombinaciones efectivas procesadas (≈): ", iter, "\n")



1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99  100  101  102  103  104  105  106  107  108  109  110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  129  130  131  132  133  134  135  136  137  138  139  140  141  142  143  144  145  146  147  148  149  150  151  152  153  154  155  156  157  158  159  160  161  162  163  164  165  166  167  168  169  170  171  172  173  174  175  176  177  178  179  180  181  182  183  184  185  186  187  188  189  190  191  192  193  194  195  196  197  198  199  200  201  202  203  204  205  206  207  208  209  210  211  212  213  214  215  216  217  218  219  220  221  222

In [ ]:
# cantidad de registros de la tabla
nrow(tb_grid_search_detalle)

[1] 575

In [ ]:
# muestro la tabla
tb_grid_search_detalle

semilla,cp,maxdepth,minsplit,minbucket,ganancia_test
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
857221,-0.10,6,50,5,41910000
857221,-0.10,6,50,10,44750000
857221,-0.10,6,50,20,42870000
857221,-0.10,6,100,5,42530000
857221,-0.10,6,100,10,44520000
857221,-0.10,6,100,20,42600000
857221,-0.10,6,100,30,41180000
857221,-0.10,6,100,40,39560000
857221,-0.10,6,200,5,42530000


In [ ]:
# genero y grabo el resumen
tb_grid_search <- tb_grid_search_detalle[,
  list( "ganancia_mean" = mean(ganancia_test),
    "qty" = .N ),
  list( cp, maxdepth, minsplit, minbucket )
]


In [ ]:
# ordeno descendente por ganancia
setorder( tb_grid_search, -ganancia_mean )


In [ ]:
# veo los 10 mejores hiperparámetros
tb_grid_search[1:10]

cp,maxdepth,minsplit,minbucket,ganancia_mean,qty
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
0.000,12,400,5,49330000,1
0.000,14,400,5,49330000,2
0.000,14,600,5,49330000,1
0.000,6,400,5,49300000,1
0.000,8,400,5,49300000,1
0.000,10,400,5,49300000,1
-0.100,8,400,10,47070000,1
-0.050,8,400,10,47070000,1
-0.010,8,400,10,47070000,1


In [ ]:
# genero un id a la tabla
tb_grid_search[, id := .I ]

fwrite( tb_grid_search,
  file = "gridsearch.txt",
  sep = "\t"
)


# 4.  Análisis de resultados de Grid Search

La salida de la corrida anterior queda en ~/buckets/b1/exp/HT2900  que corresponde a su Google Drive
<br>HT significa Hyperparameter Tuning
<br>El Grid Search es un método de fuerza bruta de un altísimo costo computacional.
<br>Queremos ver si es posible crear un algoritmo de optimización de hiperparámetros que se ahorre recorrer ciertas porciones muy malas del espacio de búsqueda. Algo del estilo “cada vez que pruebo una combinación de hiperparámetros donde  cp > 1 , la ganancia es muy mala, con lo cual ni vale la pena perder el tiempo explorando en esa region”


<br>Levante el archivo de salida gridsearch.txt  a una planilla tipo Excel y analícelo detenidamente
<br>Ordene por ganancia_mean descendente
<br>
<br>El de mayor ganancia_mean  decimos que es el primero del ranking
En Zulip, correspondiente channel  #Tarea Hogar 02 , topic Analisis Grid Search   intente contestar estas preguntas:

* ¿Qué combinaciones de hiperparámetros poseen una ganancia muy buena?
* ¿Hay algun hiperparámetro que para cierto valor siempre genera una ganancia muy mala, a independientemente de lo que valgan los otros hiperparámetros ?
* ¿Que combinaciones de hiperparámetros es pésima y hubiera sido bueno ahorrarse esas corridas ?

( tiempo estimado 30 minutos, dificultad media )

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Definí la ruta a tu archivo en Drive:
PATH_FILE = "/content/drive/MyDrive/labo1/exp/HT2900/gridsearch.txt"
# o, si querés usar el otro:
# PATH_FILE = "/content/drive/MyDrive/exp/HT2900/gridsearch_detalle.txt"


In [ ]:
!pip install plotly pandas
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio
from IPython.display import IFrame, HTML, display


In [ ]:
df = pd.read_csv(PATH_FILE, sep="\t")

# Aseguramos los tipos numéricos
for col in ["cp","maxdepth","minsplit","minbucket","ganancia_mean","qty"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Agregamos combinaciones (si hay múltiples corridas por semilla)
agg = (
    df.groupby(["cp","maxdepth","minsplit","minbucket"], as_index=False)
      .agg(ganancia_mean=("ganancia_mean","mean"),
           ganancia_sd=("ganancia_mean","std"),
           n_runs=("ganancia_mean","size"))
)

cp_levels    = sorted(agg["cp"].dropna().unique().tolist())
depth_levels = sorted(agg["maxdepth"].dropna().unique().tolist())

print(f"Archivo leído correctamente. Total de combinaciones: {len(agg)}")


In [ ]:

# ==== DASHBOARD HTML con FIX de visualización en Colab ====
import os, shutil
from IPython.display import IFrame, HTML, display
import plotly.io as pio

# 1) Generar el HTML del dashboard (inline)
html3d   = pio.to_html(fig3d, include_plotlyjs=False, full_html=False)
htmlpara = pio.to_html(parcoords, include_plotlyjs=False, full_html=False)

html_page = f"""
<!DOCTYPE html>
<html lang='es'>
<head>
<meta charset='utf-8'>
<title>Dashboard Grid Search</title>
<script src='https://cdn.plot.ly/plotly-latest.min.js'></script>
<style>
  body {{ font-family: Arial, sans-serif; margin: 20px; }}
  h1 {{ margin-bottom: 0; }}
  .caption {{ color:#555; margin-bottom: 20px; }}
  .section {{ margin-top: 26px; }}
</style>
</head>
<body>
<h1>Dashboard — Grid Search (Árbol de decisión)</h1>
<p class="caption">Menú del 3D para filtrar por <b>cp</b>. En Parallel Coordinates arrastrá en los ejes para filtrar rangos.</p>

<div class="section">
  <h2>3D interactivo</h2>
  {html3d}
</div>

<div class="section">
  <h2>Parallel Coordinates</h2>
  {htmlpara}
</div>

<hr/>
<p class="caption">Generado automáticamente.</p>
</body>
</html>
"""

# 2) Guardar a /content (local) y mostrar desde allí
LOCAL_FOLDER = "/content"  # <- servido por /files
os.makedirs(LOCAL_FOLDER, exist_ok=True)
LOCAL_DASH = os.path.join(LOCAL_FOLDER, "gridsearch_dashboard_interactivo.html")
LOCAL_3D   = os.path.join(LOCAL_FOLDER, "gridsearch_3d_interactivo.html")
LOCAL_PAR  = os.path.join(LOCAL_FOLDER, "gridsearch_parcoords_interactivo.html")

with open(LOCAL_DASH, "w", encoding="utf-8") as f:
    f.write(html_page)

# (Por si querés los individuales también en /content)
pio.write_html(fig3d,    file=LOCAL_3D, include_plotlyjs="cdn", full_html=True)
pio.write_html(parcoords,file=LOCAL_PAR, include_plotlyjs="cdn", full_html=True)

# 3) Copiar a Drive para conservarlos
OUTPUT_FOLDER = "/content/drive/MyDrive/labo1/exp/HT2900/"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
shutil.copy2(LOCAL_DASH, os.path.join(OUTPUT_FOLDER, "gridsearch_dashboard_interactivo.html"))
shutil.copy2(LOCAL_3D,   os.path.join(OUTPUT_FOLDER, "gridsearch_3d_interactivo.html"))
shutil.copy2(LOCAL_PAR,  os.path.join(OUTPUT_FOLDER, "gridsearch_parcoords_interactivo.html"))

# 4) Mostrar: A) Inline (siempre funciona)
display(HTML("<h3>Dashboard generado correctamente (inline):</h3>"))
display(HTML(html_page))

# 4) Mostrar: B) IFrame servido por Colab desde /content (evita localhost:8080)
display(HTML("<h3>Dashboard (IFrame desde /content):</h3>"))
display(IFrame(src=f"/files{LOCAL_DASH}", width="100%", height="640"))

print("Archivos guardados en Drive:")
print(os.path.join(OUTPUT_FOLDER, "gridsearch_dashboard_interactivo.html"))
print(os.path.join(OUTPUT_FOLDER, "gridsearch_3d_interactivo.html"))
print(os.path.join(OUTPUT_FOLDER, "gridsearch_parcoords_interactivo.html"))


In [ ]:
# ==== DESCARGAR DASHBOARD Y GRÁFICOS HTML ====
from google.colab import files

# Rutas locales generadas en la celda anterior
files.download("/content/gridsearch_dashboard_interactivo.html")
files.download("/content/gridsearch_3d_interactivo.html")
files.download("/content/gridsearch_parcoords_interactivo.html")

print("✅ Archivos descargados correctamente. También se guardaron en tu Drive dentro de /labo1/exp/HT2900/")
